In [1]:
# Find-S Algorithm
def findS(examples):
    hypothesis = examples[0].copy()

    for example in examples:
        if example[-1] == "Yes":
            for i in range(len(hypothesis)-1):
                if example[i] != hypothesis[i]:
                    hypothesis[i] = "?"
    return hypothesis

data = [
    ["Sunny", "Warm", "Normal", "Strong", "Warm", "Same", "Yes"],
    ["Sunny", "Warm", "High", "Strong", "Warm", "Same", "Yes"],
    ["Rainy", "Cold", "High", "Strong", "Warm", "Change", "No"]
]

print("Final Hypothesis:", findS(data))


Final Hypothesis: ['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same', 'Yes']


In [5]:
def candidate_elimination(data):
    S = data[0][:-1].copy()  
    G = [["?" for _ in S]]

    for example in data:
        attrs, output = example[:-1], example[-1]

        if output == "Yes":
            for i in range(len(S)):
                if S[i] != attrs[i]:
                    S[i] = "?"
            G = [g for g in G if all(g[i] == "?" or g[i] == attrs[i] for i in range(len(g)))]

        else:  
            new_G = []
            for i in range(len(S)):
                if S[i] != attrs[i]:
                    g = ["?" for _ in S]
                    g[i] = S[i]
                    new_G.append(g)
            G = new_G
    return S, G
data = [
    ["Sunny", "Warm", "Normal", "Strong", "Warm", "Same", "Yes"],
    ["Sunny", "Warm", "High", "Strong", "Warm", "Same", "Yes"],
    ["Rainy", "Cold", "High", "Strong", "Warm", "Change", "No"]
]

print("Specific:", candidate_elimination(data))

Specific: (['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same'], [['Sunny', '?', '?', '?', '?', '?'], ['?', 'Warm', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', '?'], ['?', '?', '?', '?', '?', 'Same']])


In [6]:
from math import log2
import pandas as pd

def entropy(col):
    values = col.value_counts(normalize=True)
    return -sum(p * log2(p) for p in values)

def info_gain(df, attr, target):
    total_entropy = entropy(df[target])
    vals = df[attr].unique()
    weighted = 0

    for v in vals:
        subset = df[df[attr] == v]
        weighted += (len(subset)/len(df)) * entropy(subset[target])
    return total_entropy - weighted

# Example dataset
df = pd.DataFrame({
    "Outlook": ["Sunny", "Rainy", "Sunny"],
    "Wind": ["Strong", "Weak", "Strong"],
    "Play": ["No", "Yes", "Yes"]
})

for col in df.columns[:-1]:
    print(col, info_gain(df, col, "Play"))


Outlook 0.2516291673878229
Wind 0.2516291673878229


In [7]:
def heuristic(hyp, example):
    return sum(hyp[i] == example[i] or hyp[i] == "?" for i in range(len(hyp)))

hypotheses = [
    ["Sunny","Warm","?","Strong","Warm","Same"],
    ["?","Warm","?","Strong","Warm","Same"],
    ["Sunny","?","?","Strong","Warm","Same"]
]

example = ["Sunny","Warm","Normal","Strong","Warm","Same"]

scores = [(h, heuristic(h, example)) for h in hypotheses]
print(scores)


[(['Sunny', 'Warm', '?', 'Strong', 'Warm', 'Same'], 6), (['?', 'Warm', '?', 'Strong', 'Warm', 'Same'], 6), (['Sunny', '?', '?', 'Strong', 'Warm', 'Same'], 6)]


#CO2

In [1]:
import numpy as np

X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([0,0,0,1])

weights = np.zeros(2)
bias = 0
lr = 0.1

def activation(z):
    return 1 if z >= 0 else 0

for epoch in range(10):
    for i in range(len(X)):
        z = np.dot(X[i], weights) + bias
        y_pred = activation(z)
        error = y[i] - y_pred
        weights += lr * error * X[i]
        bias += lr * error

print("Weights:", weights)
print("Bias:", bias)


Weights: [0.2 0.1]
Bias: -0.20000000000000004


In [2]:
import numpy as np

X = np.array([[0,0],[0,1],[1,0],[1,1]])
y = np.array([[0],[1],[1],[0]])

np.random.seed(1)
W1 = np.random.rand(2,2)
W2 = np.random.rand(2,1)
lr = 0.1

def sigmoid(x):
    return 1/(1+np.exp(-x))

def sigmoid_derivative(x):
    return x*(1-x)

for epoch in range(10000):
    h = sigmoid(np.dot(X, W1))
    out = sigmoid(np.dot(h, W2))

    error = y - out
    d_out = error * sigmoid_derivative(out)
    d_h = d_out.dot(W2.T) * sigmoid_derivative(h)

    W2 += h.T.dot(d_out) * lr
    W1 += X.T.dot(d_h) * lr

print("Output after training:\n", out)


Output after training:
 [[0.25840124]
 [0.69139438]
 [0.69130801]
 [0.39872979]]


In [3]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

iris = load_iris()
X = iris.data
y = iris.target

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = MLPClassifier(hidden_layer_sizes=(10,10), max_iter=1000)
model.fit(X_train, y_train)

y_pred = model.predict(X_test)
print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 1.0


In [4]:
import random

population_size = 6
generations = 10
population = [random.randint(0, 31) for _ in range(population_size)]

def fitness(x):
    return x*x

for g in range(generations):
    population = sorted(population, key=fitness, reverse=True)
    parents = population[:2]
    children = []

    for _ in range(population_size - 2):
        child = random.choice(parents)
        if random.random() < 0.1:
            child = random.randint(0, 31)
        children.append(child)

    population = parents + children

print("Best Solution:", population[0])


Best Solution: 25


In [5]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
population = np.random.randint(0, 2, size=(6, X.shape[1]))

def fitness(mask):
    if np.sum(mask) == 0:
        return 0
    X_sel = X[:, mask==1]
    X_train, X_test, y_train, y_test = train_test_split(X_sel, y, test_size=0.3)
    model = DecisionTreeClassifier()
    model.fit(X_train, y_train)
    return accuracy_score(y_test, model.predict(X_test))

scores = [fitness(ind) for ind in population]
best = population[np.argmax(scores)]
print("Best Feature Mask:", best)


Best Feature Mask: [1 0 0 1]


#CO3

In [6]:
# Bayes Theorem based classification
p_yes = 0.6
p_no = 0.4
p_x_given_yes = 0.7
p_x_given_no = 0.2

p_yes_given_x = (p_x_given_yes * p_yes) / (
    p_x_given_yes * p_yes + p_x_given_no * p_no
)
p_no_given_x = 1 - p_yes_given_x

print("Posterior Probability (Yes):", p_yes_given_x)
print("Posterior Probability (No):", p_no_given_x)


Posterior Probability (Yes): 0.84
Posterior Probability (No): 0.16000000000000003


In [7]:
import numpy as np

X = np.array([4, 5, 6, 5, 4, 6, 5])

mean_mle = np.mean(X)
var_mle = np.var(X)

print("Estimated Mean:", mean_mle)
print("Estimated Variance:", var_mle)


Estimated Mean: 5.0
Estimated Variance: 0.5714285714285714


In [8]:
import numpy as np

X = np.array([[1,1],[1,0],[0,1],[0,0]])
y = np.array(['Yes','Yes','No','No'])

classes = np.unique(y)
priors = {}
likelihood = {}

for c in classes:
    X_c = X[y == c]
    priors[c] = len(X_c) / len(X)
    likelihood[c] = np.mean(X_c, axis=0)

x_test = np.array([1,1])
posteriors = {}

for c in classes:
    posteriors[c] = priors[c] * np.prod(likelihood[c] ** x_test)

print("Prediction:", max(posteriors, key=posteriors.get))


Prediction: Yes


In [9]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

model = GaussianNB()
model.fit(X_train, y_train)

y_pred = model.predict(X_test)

print("Accuracy:", accuracy_score(y_test, y_pred))


Accuracy: 0.9777777777777777


In [10]:
from sklearn.mixture import GaussianMixture
from sklearn.datasets import make_blobs

X, _ = make_blobs(n_samples=200, centers=3, random_state=42)

model = GaussianMixture(n_components=3)
model.fit(X)

labels = model.predict(X)
print("Cluster Labels:", labels[:10])


Cluster Labels: [0 2 1 1 1 1 2 1 0 1]


#CO4

In [1]:
import numpy as np
from collections import Counter

X_train = np.array([[1,2],[2,3],[3,3],[6,5],[7,7]])
y_train = np.array(['A','A','A','B','B'])
X_test = np.array([3,4])
K = 3

def euclidean(a, b):
    return np.sqrt(np.sum((a-b)**2))

distances = []
for i in range(len(X_train)):
    dist = euclidean(X_train[i], X_test)
    distances.append((dist, y_train[i]))

distances.sort(key=lambda x: x[0])
neighbors = distances[:K]
labels = [n[1] for n in neighbors]

prediction = Counter(labels).most_common(1)[0][0]
print("Predicted Class:", prediction)


Predicted Class: A


In [2]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.neighbors import KNeighborsClassifier

X, y = load_iris(return_X_y=True)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

for k in range(1, 8):
    model = KNeighborsClassifier(n_neighbors=k)
    scores = cross_val_score(model, X_train, y_train, cv=5)
    print(f"K={k}, Accuracy={scores.mean():.2f}")


K=1, Accuracy=0.95
K=2, Accuracy=0.93
K=3, Accuracy=0.95
K=4, Accuracy=0.96
K=5, Accuracy=0.97
K=6, Accuracy=0.96
K=7, Accuracy=0.97


In [3]:
import numpy as np

X = np.array([1,2,3,4,5])
y = np.array([1.2,1.9,3.0,3.9,5.1])
x0 = 3
tau = 0.5

weights = np.exp(-(X - x0)**2 / (2 * tau**2))
X_bias = np.c_[np.ones(len(X)), X]
W = np.diag(weights)

theta = np.linalg.inv(X_bias.T @ W @ X_bias) @ X_bias.T @ W @ y
prediction = theta[0] + theta[1] * x0

print("Predicted Value:", prediction)


Predicted Value: 2.9787890051301127


In [4]:
import numpy as np
from sklearn.cluster import KMeans

X = np.array([[1,2],[1,3],[2,2],[6,5],[7,6],[8,7]])
y = np.array([0,0,0,1,1,1])

kmeans = KMeans(n_clusters=2)
kmeans.fit(X)
centers = kmeans.cluster_centers_

sigma = 1.0
def rbf(x, c):
    return np.exp(-np.linalg.norm(x-c)**2 / (2*sigma**2))

Phi = np.array([[rbf(x, c) for c in centers] for x in X])
weights = np.linalg.pinv(Phi) @ y

pred = np.round(Phi @ weights)
print("Predicted Labels:", pred)


Predicted Labels: [0. 0. 0. 1. 1. 1.]


In [5]:
import numpy as np

cases = {
    'Case1': np.array([1,1]),
    'Case2': np.array([2,2]),
    'Case3': np.array([5,5])
}

solutions = {
    'Case1': 'Low',
    'Case2': 'Medium',
    'Case3': 'High'
}

query = np.array([2,1])

def similarity(a, b):
    return 1 / (1 + np.linalg.norm(a-b))

scores = {c: similarity(v, query) for c, v in cases.items()}
best_case = max(scores, key=scores.get)

print("Retrieved Case:", best_case)
print("Predicted Solution:", solutions[best_case])


Retrieved Case: Case1
Predicted Solution: Low


#CO5

In [6]:
data = [
    ['Sunny','Hot','High','Weak','No'],
    ['Sunny','Hot','High','Strong','No'],
    ['Overcast','Hot','High','Weak','Yes'],
    ['Rain','Mild','High','Weak','Yes']
]

rules = []
for row in data:
    if row[-1] == 'Yes':
        rules.append(row[:-1])

print("Learned Rules:")
for r in rules:
    print("IF", r, "THEN Yes")


Learned Rules:
IF ['Overcast', 'Hot', 'High', 'Weak'] THEN Yes
IF ['Rain', 'Mild', 'High', 'Weak'] THEN Yes


In [7]:
from sklearn.datasets import load_iris
from sklearn.tree import DecisionTreeClassifier, export_text

X, y = load_iris(return_X_y=True)
model = DecisionTreeClassifier(max_depth=3)
model.fit(X, y)

rules = export_text(model)
print("Extracted Rules:\n", rules)


Extracted Rules:
 |--- feature_3 <= 0.80
|   |--- class: 0
|--- feature_3 >  0.80
|   |--- feature_3 <= 1.75
|   |   |--- feature_2 <= 4.95
|   |   |   |--- class: 1
|   |   |--- feature_2 >  4.95
|   |   |   |--- class: 2
|   |--- feature_3 >  1.75
|   |   |--- feature_2 <= 4.85
|   |   |   |--- class: 2
|   |   |--- feature_2 >  4.85
|   |   |   |--- class: 2



In [8]:
def is_safe(temp, fuel):
    return temp < 100 and fuel > 50

example = {'temp': 80, 'fuel': 60}

if is_safe(example['temp'], example['fuel']):
    rule = "IF temperature < 100 AND fuel > 50 THEN SAFE"
    print("Learned Rule:", rule)


Learned Rule: IF temperature < 100 AND fuel > 50 THEN SAFE


In [9]:
import numpy as np

states = 5
actions = 2
Q = np.zeros((states, actions))
alpha = 0.1
gamma = 0.9

for episode in range(100):
    state = 0
    while state < states - 1:
        action = np.argmax(Q[state])
        next_state = state + 1
        reward = 1 if next_state == states - 1 else 0
        Q[state, action] += alpha * (reward + gamma * np.max(Q[next_state]) - Q[state, action])
        state = next_state

print("Q-Table:\n", Q)


Q-Table:
 [[0.7232872  0.        ]
 [0.80842464 0.        ]
 [0.89971048 0.        ]
 [0.99997344 0.        ]
 [0.         0.        ]]


In [10]:
import numpy as np

Q = np.zeros((9,4))

def next_state(state, action):
    row, col = divmod(state, 3)
    if action == 0 and row > 0: row -= 1
    if action == 1 and row < 2: row += 1
    if action == 2 and col > 0: col -= 1
    if action == 3 and col < 2: col += 1
    return row*3 + col

for _ in range(200):
    s = 0
    while s != 8:
        a = np.random.randint(4)
        ns = next_state(s, a)
        reward = 1 if ns == 8 else -0.01
        Q[s,a] += 0.1 * (reward + 0.9 * np.max(Q[ns]) - Q[s,a])
        s = ns

print("Optimized Q-Values:\n", Q)


Optimized Q-Values:
 [[0.62121641 0.70169294 0.62136463 0.701608  ]
 [0.70149103 0.79080788 0.62124379 0.79068831]
 [0.79063169 0.88984734 0.7008746  0.79044685]
 [0.62115661 0.79080199 0.70160127 0.79087551]
 [0.70097544 0.88993966 0.70133157 0.88978313]
 [0.78740317 0.99993144 0.79058746 0.88876391]
 [0.70127913 0.79063546 0.79022344 0.88990399]
 [0.78663974 0.88930847 0.78882417 0.99998971]
 [0.         0.         0.         0.        ]]
